## Чтение JSON

In [ ]:
import pandas as pd

## Метод `read_json`

Мы начнем с метода **read_json**, который позволяет нам считывать простые файлы JSON в DataFrame.

Этот метод `read_json` принимает множество параметров, как мы видели в `read_csv` и `read_excel`, таких как `filepath`, `dtype` и `encoding`.

В этом случае мы попытаемся прочитать наш JSON-файл games.json.

Этот файл содержит записи игр для PlayStation в Европе с указанием названия, цены, поставщика и жанра.

In [ ]:
games = pd.read_json('https://raw.githubusercontent.com/yakushinav/omo/main/data/games.json')

In [ ]:
games.head()

## Пример вложенного JSON

Документы JSON, как правило, не так просты для чтения, как таблицы, особенно если они имеют вложенные структуры.

In [ ]:
df = pd.read_json('https://raw.githubusercontent.com/yakushinav/omo/main/data/users.json')

In [ ]:
df.head()

Это не та структура данных, которую мы хотели, поэтому в этом случае `read_json` не может быть лучшим решением.

Нам понадобится использовать модуль Python `json` для анализа нашего файла JSON в объект словаря Python, чтобы иметь возможность индексировать этот словарь и выбирать нужные вложенные данные.

Для этого мы воспользуемся методом json.load(), который преобразует наш файл JSON в словарь Python json_dict.

In [ ]:
import json
import urllib

file_url = 'https://raw.githubusercontent.com/yakushinav/omo/main/data/users.json'


data = urllib.request.urlopen(file_url)

json_dict = json.load(data)

In [ ]:
json_dict

In [ ]:
json_dict.keys()

In [ ]:
json_dict.values()

In [ ]:
for user in json_dict['info']:
    print(user)

## Использование метода pandas `from_dict`

Когда наш словарь Python готов, мы представим еще один полезный метод pandas: from_dict().

Этот метод from_dict создаст новый DataFrame из словаря.

In [ ]:
pd.DataFrame.from_dict({'Fruits': ['Apple', 'Banana']})

### Изменение ориентации данных с помощью параметра orient

«Ориентация» данных. Если ключи переданного словаря должны быть столбцами результирующего DataFrame, передайте orient=columns (поведение по умолчанию). В противном случае, если ключи должны быть строками, используйте orient=index.

In [ ]:
pd.DataFrame.from_dict({'Fruits': ['Apple', 'Banana']}, orient='index')

Используем словарь json_dict для создания нового DataFrame, но на этот раз с использованием атрибута value:

In [ ]:
df = pd.DataFrame.from_dict(json_dict)

In [ ]:
df.head()

In [ ]:
df = pd.DataFrame.from_dict(json_dict['info'])

In [ ]:
df.head()

## Использование json_normalize для сложной вложенности

В нашем примере у нас все еще есть два более сложных столбца: «адрес» и «компания».

- `адрес`: список словарей (записей)
- `компания`: словарь (запись)

Метод json_normalize будет полезен для простой распаковки и выравнивания этих данных.

Давайте распакуем столбец работ в отдельный фрейм данных. Мы также возьмем плоские столбцы, чтобы провести анализ.

In [ ]:
df.head()

In [ ]:
json_dict

In [ ]:
users = pd.json_normalize(json_dict)

users

In [ ]:
users = pd.json_normalize(json_dict['info'])

In [ ]:
users.head(3)

Теперь столбец «company» был разделен на три новых столбца: «company.name», «company.catchPhrase» и «company.bs». 

### Пользовательский разделитель для новых столбцов

Мы можем использовать параметр `sep`, чтобы определить, как будут разделены вложенные записи.

In [ ]:
users = pd.json_normalize(json_dict['info'],
                       sep='_')

In [ ]:
users.head(3)

### Распаковать столбец со списком записей

Когда у нас есть столбец со списком вложенных значений (записей, словаря) с этим сложно разобраться.

`json_normalize` позволяет нам определить параметр `record_path`, чтобы определить путь к столбцу со списком записей для их распаковки. Если этот параметр не передан, данные будут считаться массивом записей.

В нашем случае мы распакуем столбец «адрес» в отдельный DataFrame.

In [ ]:
address = pd.json_normalize(json_dict['info'],
                         sep='_',
                         record_path='address')

In [ ]:
address.head()

Параметр `record_path` может получать список столбцов, указывающих вложенный путь.

In [ ]:
address = pd.json_normalize(json_dict,
                         sep='_',
                         record_path=['info', 'address'])

In [ ]:
address.head()

### Добавление дополнительных столбцов к распакованным столбцам

Есть еще один полезный параметр «meta», который позволяет нам добавлять поля в качестве метаданных для каждой записи в результирующем «DataFrame».

В нашем случае мы добавим идентификатор пользователя к каждому адресу.

In [ ]:
address = pd.json_normalize(json_dict['info'],
                         sep='_',
                         record_path='address',
                         meta=['id'])

In [ ]:
address.head()

## Сохранить в файл JSON

Наконец, мы можем сохранить наш DataFrame как файл JSON.

In [ ]:
users.head()

Мы можем просто сгенерировать строку JSON из нашего DataFrame:

In [ ]:
users.to_json()

Или укажите путь к файлу, в котором мы хотим сохранить сгенерированный JSON-код:

In [ ]:
users.to_json('out.json')

In [ ]:
pd.read_json('out.json').head()